In [1]:
import torch
import torch.nn as nn


In [2]:
# load model
model = torch.load('./model/model.pt')


In [5]:
for param in model:
    print(param)

embedding.weight
q_matrix.weight
k_matrix.weight
v_matrix.weight
mlp.0.weight
mlp.0.bias
mlp.2.weight
mlp.2.bias


In [9]:
print(model['embedding.weight'].size())

torch.Size([11, 32])


In [10]:
import copy

In [12]:
# copy embedding
embedding = copy.deepcopy(model['embedding.weight'])
# copy qkv
q = copy.deepcopy(model['q_matrix.weight'])
k = copy.deepcopy(model['k_matrix.weight'])
v = copy.deepcopy(model['v_matrix.weight'])

In [13]:
# get qs and ks for each token
t_qs = torch.matmul(embedding, q.transpose(0, 1))
t_ks = torch.matmul(embedding, k.transpose(0, 1))


In [14]:
t_ks.size()

torch.Size([11, 32])

In [17]:
input_seq = [9, 5, 1, 6, 2, 7, 3, 8, 0, 0]
# compute attention
q_s = [t_qs[i] for i in input_seq]
k_s = [t_ks[i] for i in input_seq]

t_att = torch.matmul(torch.stack(q_s), torch.stack(k_s).transpose(0, 1))
print(t_att.size())

torch.Size([10, 10])


In [18]:
# get v
t_vs = torch.matmul(embedding, v.transpose(0, 1))
t_vs.size()

torch.Size([11, 32])

In [19]:
v_s = [t_vs[i] for i in input_seq]
v_s = torch.stack(v_s)
v_s.size()

torch.Size([10, 32])

In [20]:
print(t_att)

tensor([[ -5.3959,  -2.9583,  -2.8686,  -2.5149,  -1.1299,  -0.0956,  -2.6343,
          -3.0492,   0.0000,   0.0000],
        [ -2.7484,  -6.9906,  -8.2938,  -4.3239,  -9.4224,   2.6181,   7.0164,
           1.0293,   0.0000,   0.0000],
        [ -4.2370,  -0.1084,  -5.9481,  -5.5022,  -6.7465,   1.6919,   2.9127,
          -5.9568,   0.0000,   0.0000],
        [ -2.9856,  -1.3564,   0.2384, -10.7287,  -8.0487,   4.9876,   6.5383,
          -5.2695,   0.0000,   0.0000],
        [ -2.6035,   2.2747,  -2.9804,  -4.2712,  -6.6119,   9.1776,   6.0342,
          -4.5535,   0.0000,   0.0000],
        [ -5.7663,   0.6663,  -3.3468,  -1.6183,  -2.1964,   5.6808,   5.1520,
          -2.8558,   0.0000,   0.0000],
        [ -5.4674,   2.0909,  -3.1084,  -3.0876,  -1.1979,   2.3100,   6.9338,
          -5.8431,   0.0000,   0.0000],
        [-10.6975,  -3.1499,  -5.9383, -11.6151,  -9.2356,   3.3039,   4.3325,
         -11.9163,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000,

In [21]:
v_out = torch.matmul(t_att, v_s)

In [22]:
print(v_out.size())

torch.Size([10, 32])


In [23]:
v_out_new = v_out.sum(dim=1)
print(v_out_new.size())

torch.Size([10])


In [25]:
# get output
mlp_0_weight = copy.deepcopy(model['mlp.0.weight'])
mlp_0_bias = copy.deepcopy(model['mlp.0.bias'])
mlp_1_weight = copy.deepcopy(model['mlp.2.weight'])
mlp_1_bias = copy.deepcopy(model['mlp.2.bias'])


In [26]:
print (mlp_0_weight.size())
print (mlp_0_bias.size())
print (mlp_1_weight.size())
print (mlp_1_bias.size())

torch.Size([64, 32])
torch.Size([64])
torch.Size([11, 64])
torch.Size([11])


In [27]:
v_out_new_new = v_out.sum(dim=0)

In [28]:
print(v_out_new_new.size())

torch.Size([32])


In [29]:
output_1 = torch.matmul(v_out_new_new, mlp_0_weight.transpose(0, 1)) + mlp_0_bias
output_1.size()

torch.Size([64])

In [30]:
output_1_relu = nn.functional.relu(output_1)

In [31]:
output_2 = torch.matmul(output_1_relu, mlp_1_weight.transpose(0, 1)) + mlp_1_bias

In [32]:
print(output_2.size())

torch.Size([11])


In [33]:
print(output_2)

tensor([ -5.0723,   8.9644,  14.0118,  30.5182, -21.5813,   1.5899,  19.3786,
         13.1759, -19.6485, -18.1134, -44.3425])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)